# Walmart sales

This project's goals are to build a  machine learning model that is able to estimate the weekly sales in Walmart stores, with the best precision possible on the predictions made. Such a model would help them understand better how the sales are influenced by economic indicators, and might be used to plan future marketing campaigns.

### Import packages

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.templates["jedha"] = go.layout.Template(
    layout_colorway=["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]
)
pio.templates.default = "jedha"

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Import dataset

In [2]:
data = pd.read_csv("Walmart_Store_sales.csv")

### Exploratory data analysis

In [3]:
print(f"Number of rows : {data.shape[0]}")
print()

print("Display of dataset: ")
display(data.head())
print()

print("Basics statistics: ")
data_desc = data.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*data.isnull().sum()/data.shape[0])

Number of rows : 150

Display of dataset: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092



Basics statistics: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,07-01-2011,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000



Percentage of missing values: 


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

There is quite a bit of missing data for some variables ! 

## Data specifics

- __Store__  = store number, is not a unique identifier could be used after dummy encoding (because not ordinal variable), but it'll add 19 columns for 150 rows

- __Date__ cannot be used as is and displays 12% missing data. Although I could create new columns: Year, month, etc..

- __Weekly_sales__ is the target variable Y : 9% missing, I need to remove those lines.

- __Holiday_Flag__ 1 == Week is a holiday, 0 == Week is not a holiday. 8% missing data, probably not holidays, replace with zeros

- __Temperature__ in Fahrenheits, 12% missing data, impute with mean

- __Fuel_Price__ 9.3% missing data, impute with mean

- __CPI__ customer price index, measures inflation, impute with mean

- __Unemployment__ unemployement index, impute with mean

In [4]:
from plotly.subplots import make_subplots


fig = make_subplots(rows=2, cols=3, subplot_titles=("CPI", "Fuel_Price", "Weekly_Sales", "Temperature", "Unemployment","Holiday_Flag"))

fig.add_trace(go.Histogram(x=data.loc[:,'CPI']), row = 1, col = 1)
fig.add_trace(go.Histogram(x=data.loc[:,'Fuel_Price']), row = 1, col = 2)
fig.add_trace(go.Histogram(x=data.loc[:,'Weekly_Sales']), row = 1, col = 3)
fig.add_trace(go.Histogram(x=data.loc[:,'Temperature']), row = 2, col = 1)
fig.add_trace(go.Histogram(x=data.loc[:,'Unemployment']), row = 2, col = 2)
fig.add_trace(go.Histogram(x=data.loc[:,'Holiday_Flag']), row = 2, col = 3)

fig.update_layout(showlegend=False,
                  height = 600,
                  width = 1000)

fig.show()

_We can see that CPI has a bimodal distribution, has does fuel price. Weekly sales, temperature and unemployment are more evnly distributed, but unemployment has visible outliers. Holiday flag is a binary variable_

In [5]:
data['Date'] = pd.to_datetime(data['Date'],format='%d-%m-%Y')

fig = make_subplots(rows=3, cols=2, subplot_titles=("Store", "CPI", "Unemployment", "Temperature", "Fuel_Price", "Date"))

fig.add_trace(go.Scatter(x=data['Store'], y=data['Weekly_Sales'], mode='markers'), row = 1, col = 1)
fig.add_trace(go.Scatter(x=data['CPI'], y=data['Weekly_Sales'], mode='markers'), row = 1, col = 2)
fig.add_trace(go.Scatter(x=data['Unemployment'], y=data['Weekly_Sales'], mode='markers'), row = 2, col = 1)
fig.add_trace(go.Scatter(x=data['Temperature'], y=data['Weekly_Sales'], mode='markers'), row = 2, col = 2)
fig.add_trace(go.Scatter(x=data['Fuel_Price'], y=data['Weekly_Sales'], mode='markers'), row = 3, col = 1)
fig.add_trace(go.Scatter(x=data['Date'], y=data['Weekly_Sales'], mode='markers'), row = 3, col = 2)


fig.update_layout(showlegend=False,
                  height = 600,
                  width = 1000)

fig.show()


_Stores show great variation among them in terms of weekly_sales, this is an interesting feature to consider later on. Aside from this, weekly sales maybe positiveley correlated with unemployment and negatively correlated with CPI, let's verify with a correlation matrix_

In [6]:
corr_matrix = data.corr().round(2)

import plotly.figure_factory as ff

fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist(),
                                  colorscale = 'Teal')

fig.update_layout(autosize=False, height=800, width = 1000,
                  margin = {'l': 120})
fig.show()

_Indeed, Weekly sales is correlated with CPI and Unemployment but also with every other variable to different degrees._

## Feature selection


__Create new columns: Year, month, day__

In [7]:
data['Date'] = pd.to_datetime(data['Date'],format='%d-%m-%Y')

data["Year"] = data['Date'].apply(lambda x : int(x.strftime("%Y")) if pd.notnull(x) else None)
data["Month"] = data['Date'].apply(lambda x : int(x.strftime("%m")) if pd.notnull(x) else None)
data["Day"] = data['Date'].apply(lambda x : int(x.strftime("%d")) if pd.notnull(x) else None)

## Avoid day of week because it is always 4

data = data.drop('Date', axis=1) # Drop date column

__Drop missing target values__

In [8]:
missing_y = data[data['Weekly_Sales'].isnull()].index.to_list()
data = data.drop(missing_y)

__Remove outliers with values +/- 3 standard deviations for Temperature, Fuel_Price, CPI and Unemployment__

In [9]:
cols = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
cols

for col in cols :
    low = data[col].mean() - (data[col].std()*3)
    high = data[col].mean() + (data[col].std()*3)
    to_remove = data[(data[col] < low) | (data[col] > high)].index.to_list()
    data = data.drop(to_remove)

print(len(data), "lines have been retained")

131 lines have been retained


__Separate target and features__

In [10]:
target_variable = 'Weekly_Sales'

Y = data.loc[:,target_variable]
X = data.drop(target_variable, axis=1)

## Linear regression

__Train / Test split__

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=0)

__Separate numerical and categorical features, and apply additional preprocessing__

_Numerical variables : imputation of missing values with the mean + normalization_

_Categorical variables : Imputation with most frequent + One Hot encoding_

In [12]:
num_cols = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month', 'Day']

num_transform = Pipeline(steps=[
         ('imputer', SimpleImputer(strategy='mean')),
         ('scaler', StandardScaler())
     ])

In [13]:
cat_cols = ['Store', 'Holiday_Flag']

cat_transform = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='first'))
    ])

In [14]:
preprocess = ColumnTransformer(transformers=[
    ('num', num_transform, num_cols),
    ('cat', cat_transform, cat_cols)
])

In [15]:
print("Starting preprocess on train set: ")
print(X_train.head())
X_train = preprocess.fit_transform(X_train)
print('Done.')
print(X_train[0:5])

print("Starting preprocess on test set: ")
print(X_test.head())
X_test = preprocess.transform(X_test)
print('Done.')
print(X_train[0:5])


Starting preprocess on train set: 
     Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
136    4.0           NaN        84.59       3.469  129.112500         5.644   
78     1.0           0.0        62.25       3.308  218.220509         7.866   
17    18.0           0.0        21.33       2.788  131.527903         9.202   
108   18.0           0.0        69.12       2.906  132.293936           NaN   
141    5.0           0.0        62.37         NaN  212.560411         6.768   

       Year  Month   Day  
136  2011.0    7.0   8.0  
78   2011.0   11.0  18.0  
17      NaN    NaN   NaN  
108  2010.0    5.0  28.0  
141  2010.0   11.0  12.0  
Done.
[[ 1.47125449  0.34231519 -1.33363663 -1.83554421  0.19302754  0.21122551
  -1.15780865  0.          0.          1.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.14671106 -0.015658

__Train Linear model__

In [16]:
Linear = LinearRegression()

print("Training model...")
Linear.fit(X_train, Y_train) # Training is always done on train set !!
print("...Done.")

Training model...
...Done.


__Display R2 scores for Train and Test__

In [17]:
print("R2 score on training set : ", Linear.score(X_train, Y_train))
print("R2 score on test set : ", Linear.score(X_test, Y_test))

R2 score on training set :  0.9731935046590678
R2 score on test set :  0.9344654257543309


_R2 scores are very high! However, the difference between each score might be indicative of overfitting._

__10-fold Cross validation__

In [18]:
regressor = LinearRegression()
scores = cross_val_score(regressor, X_train, Y_train, cv=5)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

The cross-validated R2-score is :  0.944867942090513
The standard deviation is :  0.0193298011921228


_Cross Validation results show that R2 scores can vary by +/- 0.01. The variation between Train and Test sets is greater that this, which means that the model is probably overfitting_

In [19]:
names = [x[5:] for x in preprocess.get_feature_names_out().tolist()]

coeffs = pd.DataFrame(index = names, data = Linear.coef_.transpose(), columns=['coefficients'])
feature_importance = coeffs.sort_values(by='coefficients')

fig = px.bar(feature_importance)
fig.update_layout(showlegend = False,
                  margin = {'l': 120}
                 )

fig.update_layout(autosize = False, height = 600, width = 1200)
fig.show() 

_We can see that the Store variable has a great effect both positive and negative on weekly sales. This can be explained simply by the geographical location of each store. Location of a store in a rich or a poor neighborhood can easily explain these effects._ 

_Unemployment index negatively affect weekly sale in our model, because when people are unemployes, well, they can't spend as much!_

_Of the features that affect positively weekly_sales we can see CPI, which measures the average change over time in the prices paid by urban consumers for a market basket. What this means is that when CPI increases, the price of a merket basket is higher. Given this behaviour, people might be spending more money on the same basket, therefore increasing weekly sales._

## Regularization

__Perform regularization with Ridge and Lasso__

In [20]:
Ridge1 = Ridge()
print("Training model...")
Ridge1.fit(X_train, Y_train)
print("...Done.")

print(f"R2 score on training set: {Ridge1.score(X_train,Y_train)}")
print(f"R2 score on test set: {Ridge1.score(X_test,Y_test)}")

Training model...
...Done.
R2 score on training set: 0.9333959397773349
R2 score on test set: 0.9246045517256279


_We can see that our Ridge regression without hyperparameter optimization is worse that our original model._

In [21]:
Lasso1 = Lasso()
print("Training model...")
Lasso1.fit(X_train, Y_train)
print("...Done.")

print(f"R2 score on training set: {Lasso1.score(X_train,Y_train)}")
print(f"R2 score on test set: {Lasso1.score(X_test,Y_test)}")

Training model...
...Done.
R2 score on training set: 0.9731935021470872
R2 score on test set: 0.9344879874862412


_Lasso however is closer to our initial performance but we can improve these models with hyperparameter optimization_

__Exhaustive search over specified parameter values for an estimator__

In [22]:
ridge = Ridge()
params = {
     'alpha': np.linspace(0, 0.1, 100, endpoint=False)
}

best_ridge = GridSearchCV(ridge, param_grid=params, cv=10)
best_ridge.fit(X_train, Y_train)

print("Best hyperparameters : ", best_ridge.best_params_)
print("Best R2 score : ", best_ridge.best_score_)

Best hyperparameters :  {'alpha': 0.016}
Best R2 score :  0.9406619083478075


In [23]:
lasso = Lasso()
params = {
     'alpha': np.linspace(1, 1000, 100, endpoint=False)
}

best_lasso = GridSearchCV(lasso, param_grid=params, cv=10)
best_lasso.fit(X_train, Y_train)

print("Best hyperparameters : ", best_lasso.best_params_)
print("Best R2 score : ", best_lasso.best_score_) 

Best hyperparameters :  {'alpha': 460.54}
Best R2 score :  0.9419905631074263


_After hyperparameter optimization we can compare the regularization models to the intitial classical linear regression_

In [24]:
scores = cross_val_score(best_lasso, X_train, Y_train, cv=5)
print("Lasso:")
print(f"R2 score on training set: {best_lasso.score(X_train,Y_train)}")
print(f"R2 score on test set: {best_lasso.score(X_test,Y_test)}")
print('Cross Validation standard deviation is : ', scores.std())

print()

scores = cross_val_score(best_ridge, X_train, Y_train, cv=5)
print("Ridge:")
print(f"R2 score on training set: {best_ridge.score(X_train,Y_train)}")
print(f"R2 score on test set: {best_ridge.score(X_test,Y_test)}")
print('Cross Validation standard deviation is : ', scores.std())


print()

scores = cross_val_score(Linear, X_train, Y_train, cv=5)
print("Linear regression:")
print(f"R2 score on training set: {Linear.score(X_train,Y_train)}")
print(f"R2 score on test set: {Linear.score(X_test,Y_test)}")
print('Cross Validation standard deviation is : ', scores.std())

Lasso:
R2 score on training set: 0.9727210447748377
R2 score on test set: 0.9432760024236482
Cross Validation standard deviation is :  0.025762212497012008

Ridge:
R2 score on training set: 0.9731486924499261
R2 score on test set: 0.9366489345216543
Cross Validation standard deviation is :  0.019507168249811625

Linear regression:
R2 score on training set: 0.9731935046590678
R2 score on test set: 0.9344654257543309
Cross Validation standard deviation is :  0.0193298011921228


- We can see that the differences bewteen train and test sets is still present in our three models. It is similar between the Linear and the Ridge models, but it is lower in the Lasso model.

- Regarding the standard deviation from the cross validation again they are similar bewteen the Linear and the Ridge models, and it is higher in the Lasso model.

- These results reflect the trade-off bewteen bias and variance in regression models !

- However, given these results, I would choose the Lasso regression because it is better at reducing the overfitting present in the initial linear regression than the ridge model.

- The Lasso model might still be overfitting, but maybe additional data and feature selection might help to further reduce it

- Let's compare feature importance to visualise the effect of the models on feature coefficients.

In [25]:
names = [x[5:] for x in preprocess.get_feature_names_out().tolist()]

coeffs = pd.DataFrame(data = {'Features' : names , 'Ridge' : best_ridge.best_estimator_.coef_.transpose(),'Lasso' : best_lasso.best_estimator_.coef_.transpose(), 'Linear' : Linear.coef_.transpose()})
coeffs = coeffs.sort_values(by='Ridge')

coeffs = pd.melt(coeffs, id_vars="Features", var_name ="Method")

fig = px.bar(coeffs, x='Features', y='value', color='Method')
fig.update_layout(margin = {'l': 120})
fig.update_layout(barmode='group')
fig.update_layout(autosize = False, height = 600, width = 1200)
fig.show()

_Both the Ridge and Lasso models diminished the amplitude of some of the features, and the Lasso model did set one of them to zero._

__Overall the models perform very well, and regularization barlely attenuates the small overfitting of the original linear regression !__